<h1>Sara Oppenheim, capstone project!</h1>

<h1>Pests, pesticides, production and profits: soybeans in the USA 1965-2021.</h1>

<h4>Import packages</h4>

In [ ]:
import numpy as np
import pandas as pd
#import geopandas as gp 
#when I try to install geopandas I get fatal inconsistency errors from conda
import plotly as ply
import plotly.express as px
import plotly.io as pio
import kaleido as kaleido
import scipy as sc
import sympy as sy
import matplotlib as matplot
import matplotlib.pyplot as plt
import statistics as stat
import seaborn as sns
%matplotlib inline


<h3>Convert data to dataframes</h3>

In [ ]:
#Create a dataframe from each of the data sets. 
#this list includes the na indicators across all of the files
missing_values = ["(null)"," ","na","NR","Nan","(NA)","NaN"]

#this is the current distribution of the soybean aphid in the US
#source CABI Plant Pest Distribution database: https://www.cabi.org/isc/datasheet/6203#todistributionDatabaseTable
soyaphiddist=pd.read_csv("US_soy_aphid_distribution2.csv",na_values=missing_values)

#this is the annual costs and returns data for soybean
#data were split by "historical" vs "recent." I merge the two data sets before importing
#source historical: https://www.ers.usda.gov/data-products/commodity-costs-and-returns/commodity-costs-and-returns/#Historical%20Costs%20and%20Returns:%20Soybeans
#source recent: https://www.ers.usda.gov/data-products/commodity-costs-and-returns/commodity-costs-and-returns/#Recent%20Cost%20and%20Returns
soycostreturn=pd.read_csv("US_soybean_costs_and_returns_1975-2021.csv",na_values=missing_values)

#this data set shows the percent of US soybean treated with insecticides or herbicides
#source https://www.ers.usda.gov/publications/pub-details/?pubid=43855
pctreated=pd.read_csv("soy_pct_treated_acres.csv")

#this is data from the US Register of Introduced and Invasive Species data
#source https://www.sciencebase.gov/catalog/item/6144f1ccd34e0df5fb95b5cb
invasives=pd.read_csv("US_registry_of_introduced_and_invasive_species.csv",na_values=missing_values)

#this is data on acres planted, pesticide use, costs, and returns summed across 21 Selected Crops from 1960-2008
#the crops included are Apples, Barley, Corn, Cotton, Grapefruit, Grapes, Lemons, Lettuce, Oranges, Peaches, Peanuts, Pears, Pecans, Potatoes, Rice, Sorghum, Soybeans, Sugarcane, Sweetcorn, Tomatoes, and Wheat
#source https://www.ers.usda.gov/publications/pub-details/?pubid=43855
pesticides21crops=pd.read_csv("21_crops_value_and_cost_1960-2008.csv",na_values=missing_values)

#this data describes non-chemical pest control measures in soybean from 1991-2020
#source USDA National Agricultural Statistics Service https://quickstats.nass.usda.gov/
nassnonchemcontrol=pd.read_csv("nass_soy_pest_non_chem_control.csv",na_values=missing_values)

#this data describes acres of soybean planted, yield per acre, and sale price from 1924-2022
#source https://quickstats.nass.usda.gov/results/AA292938-D6B5-3B0D-B779-64A46043B700
nassyield=pd.read_csv("nass_soy_planted_yield_and_price.csv",na_values=missing_values)

#this is annual production data (1993-2022) for field crops in the US, which includes corn, soybean, grains, etc.
#source https://quickstats.nass.usda.gov/results/47D5D9C1-AD06-3A62-8B45-B09DEC5F7D8F
fieldcrops=pd.read_csv("NASS_field_crops_data.csv",na_values=missing_values)

#this is annual pesticide use data for soybean
#source https://hygeia-analytics.com/pesticides/usage/puds-the-pesticide-use-data-system/
insecticidesbyyear=pd.read_csv("soybean_insecticides_by_year.csv",na_values=missing_values)



<h3>Examine and tidy up the data</h3>

In [ ]:
#Create a list of all the dataframes so i can loop through them for QC steps
dflist = {
    'insecticidesbyyear': insecticidesbyyear,'fieldcrops': fieldcrops, 
           'nassyield': nassyield, 'nassnonchemcontrol': nassnonchemcontrol,
           'nasschemcontrol': nasschemcontrol, 'fortyyearchange': fortyyearchange, 
           'pesticides21crops': pesticides21crops,'invasives': invasives,
           'soycostreturn': soycostreturn,'soyaphiddist': soyaphiddist
}

for i, df in dflist.items():
    print(i)
    print(df.head())
    

<h2>Insecticide use in the US over the past 30 years</h2>

This is annual pesticide use data for soybean in the US  
source https://hygeia-analytics.com/pesticides/usage/puds-the-pesticide-use-data-system/

In [ ]:
#generate missing counts and heat maps to assess missing data
print("insecticidesbyyear",sns.heatmap(insecticidesbyyear.isnull(),cbar=True,vmin=0,vmax=.3))
print(insecticidesbyyear.isnull().sum())

In [ ]:
#clean up insecticidesbyyear
#rename columns to be more informative
insecticidesbyyear.rename(columns={"year":"Year","Analyte":"Active ingredient",
                                       "Class": "Insecticide class",
                                       "PercentAcresTreated":"Fraction of acres treated with this insecticide",
                                       "RatePerCropYear": "Rate per crop year"},inplace=True)
#retain only the relevant columns
insecticidesbyyear=insecticidesbyyear[["Year","Active ingredient","Insecticide class",
                                       "Fraction of acres treated with this insecticide",
                                       "Rate per crop year"]]
#drop any rows where relevant sata are missing
insecticidesbyyear=insecticidesbyyear[insecticidesbyyear["Insecticide class"].notna()] 
insecticidesbyyear=insecticidesbyyear[insecticidesbyyear["Year"].notna()]
insecticidesbyyear=insecticidesbyyear[insecticidesbyyear["Fraction of acres treated with this insecticide"].notna()]



In [ ]:
#create separate dfs for each insecticide class in insecticidesbyyear, 
#I will need these later
#could have done "str.contains" but "==" seems more stringent
#bi=insecticidesbyyear[insecticidesbyyear["Insecticide class"].str.contains("biologic")]

py=insecticidesbyyear[insecticidesbyyear["Insecticide class"]=="pyrethroid"]
og=insecticidesbyyear[insecticidesbyyear["Insecticide class"]=="organophosphate"]
ca=insecticidesbyyear[insecticidesbyyear["Insecticide class"]=="carbamate"]
ne=insecticidesbyyear[insecticidesbyyear["Insecticide class"]=="neonicitinoid"]
be=insecticidesbyyear[insecticidesbyyear["Insecticide class"]=="benzoylurea"]
ry=insecticidesbyyear[insecticidesbyyear["Insecticide class"]=="ryanoid"]
di=insecticidesbyyear[insecticidesbyyear["Insecticide class"]=="diacylhydrazine"]
bi=insecticidesbyyear[insecticidesbyyear["Insecticide class"]=="biologic"]


**Have the kind of insecticides being used changed over time?**  

In [ ]:
#Are there overall changes in the classes of insecticides being used?
plt.title("Changes in insecticide use from 1991-2018")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide", 
              data=insecticidesbyyear,hue="Insecticide class" ,
             )
plt.show()
fig.savefig("Changes in insecticide use 1991-2018, all classes.png")

In [ ]:
#There were many outliers above, they seem to be concentrated in more recent years
#I'll look at a data set that only includes records from 2005 onward
recentiby=insecticidesbyyear[insecticidesbyyear["Year"]>=2005]
plt.figure(figsize=(12,8))
plt.title("Changes in insecticide use from 2005-2018")
sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide", 
              data=recentiby,hue="Insecticide class",
             )
plt.show()


In [ ]:
#There are still outliers and it looks like most of them are from 2 insecticide classes
#I'll look at a dataset that excludes those 2 classes
nonpyiby=insecticidesbyyear[insecticidesbyyear["Insecticide class"]!="pyrethroid"]
nonpyorgiby=nonpyiby[nonpyiby["Insecticide class"]!="organophosphate"]

plt.title("Changes in insecticide use 1991-2018, excluding pyrethroids and organophosphates")
sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide", 
              data=nonpyorgiby,hue="Insecticide class",
             )
plt.show()


In [ ]:
#It looks like the other classes all changed in a linear way over time, 
#though the shape is very different across classes 
#Trying a more complex model
plt.title("Changes in insecticide use 1991-2018, excluding pyrethroids and organophosphates")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide", 
              data=nonpyorgiby,hue="Insecticide class",
           order=3,fit_reg=True, n_boot=10000,
           x_ci="sd",ci=20,scatter_kws={"s": 80}
             )
plt.show()
fig.savefig("Changes in insecticide use 1991-2018, excluding pyrethroids and organophosphates.png")

In [ ]:
#How about the pyrethroids and organophosphates? Now that they are isolated is there a pattern?
#I'll look at a dataset that includes only those 2 classes
pyorgiby=insecticidesbyyear[(insecticidesbyyear["Insecticide class"]=="pyrethroid") |
                         (insecticidesbyyear["Insecticide class"]=="organophosphate")
                        ]
plt.title("Changes in insecticide use 1991-2018, pyrethroids and organophosphates only")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide", 
              data=pyorgiby,hue="Insecticide class",
           order=3,fit_reg=True, n_boot=10000,
           x_ci="sd",ci=20,scatter_kws={"s": 80}
             )
plt.show()
fig.savefig('Changes in insecticide use 1991-2018, pyrethroids and organophosphates only.png')


In [ ]:
#Still looks crazy, how about one at a time?
#I'll look at a dataset that has only pyrethroids
plt.title("Changes in insecticide use 1991-2018, pyrethroids only")
sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide",data=py,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
             )
plt.show()


In [ ]:
#Still looks crazy
#BUT each class has many active ingredients contributing. Maybe these are driving the pattern?
#I'll look at a dataset that has only pyrethroids
pyorgiby=pyorgiby.sort_values(by="Active ingredient",ascending=False)
plt.title("Changes in insecticide use by active ingredient 1991-2018, organophosphates pyrethroids only")
sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide",data=pyorgiby,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           hue="Active ingredient",
             )
plt.show()


In [ ]:
#Ah-ha! It looks like 3 particular ingredients are responsible for the outliers:
#Lambda-cyhalothrin (pyrethroid), Chlorpyrifos (organophosphate), and Bifenthrin(pyrethroid)
pyorgiby.groupby(["Active ingredient","Insecticide class"])["Fraction of acres treated with this insecticide"].mean().sort_values()



In [ ]:
#How do the other classes look individually?
#I'll cycle through them all
#pyrethroids
plt.title("Changes in insecticide use 1991-2018, pyrethroids only")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide",data=py,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           hue="Active ingredient",
             )
plt.show()
fig.savefig('Changes in insecticide use 1991-2018, pyrethroids only.png')


In [ ]:
#How do the other classes look individually?
#organophosphates
plt.title("Changes in insecticide use 1991-2018, organophosphates only")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide",data=og,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           hue="Active ingredient",
             )
plt.show()
fig.savefig('Changes in insecticide use 1991-2018, organophosphates only.png')


In [ ]:
#How do the other classes look individually?
#carbamates
plt.title("Changes in insecticide use 1991-2018, carbamates only")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide",data=ca,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           hue="Active ingredient",
             )
plt.show()
fig.savefig('Changes in insecticide use 1991-2018, carbamates only.png')


In [ ]:
#How do the other classes look individually?
#neonicitinoids
plt.title("Changes in insecticide use 1991-2018, neonicitinoids only")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide",data=ne,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           hue="Active ingredient",
             )
plt.show()
fig.savefig('Changes in insecticide use 1991-2018, neonicitinoids only.png')


In [ ]:
#How do the other classes look individually?
#Ibenzoylureas
plt.title("Changes in insecticide use 1991-2018, benzoylureas only")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide",data=be,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           hue="Active ingredient",
             )
plt.show()
fig.savefig('Changes in insecticide use 1991-2018, benzoylureas only.png')


In [ ]:
#How do the other classes look individually?
#ryanoids
plt.title("Changes in insecticide use 1991-2018, ryanoids only")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide",data=ry,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           hue="Active ingredient",
             )
plt.show()
fig.savefig('Changes in insecticide use 1991-2018, ryanoids only.png')


In [ ]:
#How do the other classes look individually?
#diacylhydrazines
plt.title("Changes in insecticide use 1991-2018, diacylhydrazines only")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide",data=di,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           hue="Active ingredient",
             )
plt.show()
fig.savefig('Changes in insecticide use 1991-2018, diacylhydrazines only.png')


In [ ]:
#How do the other classes look individually?
#biologics
plt.title("Changes in insecticide use 1991-2018, biologics only")
fig=sns.lmplot(x="Year", y="Fraction of acres treated with this insecticide",data=bi,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           hue="Active ingredient",
             )
plt.show()
fig.savefig('Changes in insecticide use 1991-2018, biologics only.png')


<h2>"Non-chemical" pesticide use in the US over the past 10 years</h2>

These data describes non-chemical pest control measures in soybean from 1991-2020
Source USDA National Agricultural Statistics Service https://quickstats.nass.usda.gov/

In [ ]:
#generate missing counts and heat maps to assess missing data
print("nassnonchemcontrol",sns.heatmap(nassnonchemcontrol.isnull(),cbar=True,vmin=0,vmax=.3))
print(nassnonchemcontrol.isnull().sum())


In [ ]:
#clean up the nonchem data
#print(nassnonchemcontrol.head())
nassnonchemcontrol["Commodity"]=nassnonchemcontrol["Commodity"].astype('string')
nassnonchemcontrol["Domain"]=nassnonchemcontrol["Domain"].astype('string')
nassnonchemcontrol["Class"]=nassnonchemcontrol["Class"].astype('string')
nassnonchemcontrol["PCT OF AREA PLANTED VALUE"]=pd.to_numeric(nassnonchemcontrol["PCT OF AREA PLANTED VALUE"],errors='coerce')

nassnonchemcontrol.dtypes
nassnonchemcontrol=nassnonchemcontrol[['Year', 'Commodity','Domain','Class','PCT OF AREA PLANTED VALUE']]

nassnonchemcontrol=nassnonchemcontrol.rename(columns = {'Domain':'Category','Class':'Method', 
                                     'PCT OF AREA PLANTED VALUE':'Percent of planted area treated with method'})
nassnonchemcontrol=nassnonchemcontrol.dropna()


In [ ]:
#create separate dfs for each nonchem control category
mo=nassnonchemcontrol[nassnonchemcontrol["Category"]=="PRACTICE, MONITORING"]
pr=nassnonchemcontrol[nassnonchemcontrol["Category"]=="PRACTICE, PREVENTION"]
su=nassnonchemcontrol[nassnonchemcontrol["Category"]=="PRACTICE, SUPPRESSION"]
av=nassnonchemcontrol[nassnonchemcontrol["Category"]=="PRACTICE, AVOIDANCE"]


In [ ]:
#Have nonchemical control use patterns changed over time?
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Nonchemical pest control on soybeans 2012-2020")
sns.barplot(x="Category", y="Percent of planted area treated with method", data=nassnonchemcontrol,hue="Year")
#remove legend from sns plot
#plt.legend([],[], frameon=False)
plt.show()
fig.savefig('Nonchemical pest control on soybeans 2012-2020.png')



In [ ]:
#Nonchemical
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Nonchemical pest control on soybeans 2012-2020")
fig=sns.lmplot(x="Year", y="Percent of planted area treated with method", data=nassnonchemcontrol,hue="Category",
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           #line_kws={'color': 'red'},
               order=2,
             )
plt.show()
fig.savefig('Nonchemical pest control on soybeans 2012-2020 all.png')


In [ ]:
#Have nonchemical control use patterns changed over time? monitoring
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Nonchemical pest control on soybeans 2012-2020: monitoring")
fig=sns.lmplot(x="Year", y="Percent of planted area treated with method", data=mo,hue="Method",
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           #line_kws={'color': 'red'},
               #order=2,
             )
plt.show()
fig.savefig('Nonchemical pest control on soybeans 2012-2020: monitoring.png')


In [ ]:
#Have nonchemical control use patterns changed over time? avoidance
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Nonchemical pest control on soybeans 2012-2020: avoidance")
fig=sns.lmplot(x="Year", y="Percent of planted area treated with method", data=av,hue="Method",
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           #line_kws={'color': 'red'},
               #order=2,
             )
plt.show()
fig.savefig('Nonchemical pest control on soybeans 2012-2020: avoidance.png')


In [ ]:
#Have nonchemical control use patterns changed over time? prevention
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Nonchemical pest control on soybeans 2012-2020: prevention")
fig=sns.lmplot(x="Year", y="Percent of planted area treated with method", data=pr,hue="Method",
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           #line_kws={'color': 'red'},
               #order=2,
             )
plt.show()
fig.savefig('Nonchemical pest control on soybeans 2012-2020: prevention.png')


In [ ]:
#Have nonchemical control use patterns changed over time? suppression
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Nonchemical pest control on soybeans 2012-2020: suppression")
fig=sns.lmplot(x="Year", y="Percent of planted area treated with method", data=su,hue="Method",
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           #line_kws={'color': 'red'},
               order=2,
             )
plt.show()
fig.savefig('Nonchemical pest control on soybeans 2012-2020: suppression.png')


In [ ]:
#Have nonchemical control use patterns changed over time? suppression
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Nonchemical pest control on soybeans 2012-2020: suppression")

suppression=sns.barplot(x="Year", y="Percent of planted area treated with method", data=su,hue="Method")
#suppression.set_xticklabels()
#for item in suppression.get_xticklabels():
 #   item.set_rotation(45)
#remove legend from sns plot
#plt.legend([],[], frameon=False)
plt.show()
#fig.savefig('Nonchemical pest control on soybeans 2012-2020.png')



<h2>Non-native pests in the US: "invasions" and accidental introductions</h2>

These data are from the US Register of Introduced and Invasive Species data  

Source https://www.sciencebase.gov/catalog/item/6144f1ccd34e0df5fb95b5cb

In [ ]:
#generate missing counts and heat maps to assess missing data
print("invasives",sns.heatmap(invasives.isnull(),cbar=True,vmin=0,vmax=.3))
print(invasives.isnull().sum())
invasives.shape

In [ ]:
#clean up the invasives data
invasives=invasives[['scientificName', 'vernacularName','IntroDateNumber','phylum','class','order']]
invasives=invasives.dropna()
invasives=invasives.rename(columns = {'scientificName':'scientific name','vernacularName': 'common name', 
                                      'IntroDateNumber':'year reported'})


In [ ]:
#clean up the invasives data
invasives["scientific name"]=invasives["scientific name"].astype('string')
invasives["common name"]=invasives["common name"].astype('string')
invasives["phylum"]=invasives["phylum"].astype('string')
invasives["class"]=invasives["class"].astype('string')
invasives["order"]=invasives["order"].astype('string')
invasives['year reported']=pd.to_numeric(invasives['year reported'],errors='coerce')
invasives["year reported"]=invasives["year reported"].astype('int')


In [ ]:
#which organisms are the most frequent invasives?
invasives.groupby(['phylum'])['year reported'].count()

In [ ]:
#which organisms are the most frequent invasives?
invasives["year reported"].value_counts()
plt.title("Invasive species in the US, by phylum")
invasives.groupby(['phylum'])['year reported'].count().plot(kind='bar',figsize=(12,10),xlabel='phylum of invasive species',
                                                            title='Invasive species in the US, by phylum',color="salmon",
                                                            ylabel="number invasives in phylum",width=1.2)
plt.savefig('Invasive species in the US, by phylum.png')



In [ ]:
#jointplot all invasives
plt.title("Invasive species in the US from 400 AD to the present")
sns.jointplot(x='year reported', y='class', data=invasives, kind='scatter',
             hue="phylum",height=14,palette="Accent_r",
             )

plt.savefig('Invasive species in the US from 400AD to the present, by year.png')


In [ ]:
#arthropods are the most frequent invasives. which arthropods are the biggest offenders?
ar=invasives[invasives["phylum"]=="Arthropoda"]
ar.groupby(['order'])['year reported'].count().sort_values(ascending=False)



In [ ]:
#histogram arthropods
plt.title("Invasive arthropods in the US, by taxonomic order")
ar.hist(column="year reported", by="order", grid=False,figsize=(25, 20), legend=False, 
        color='tab:pink',bins=30,histtype="stepfilled",orientation="horizontal",
        density=True,
        #range="2010.0","2022.0"
       )

plt.savefig('Invasive arthropods in the US, by individual order.png')



In [ ]:
#which arthropods are the most frequent invasives?
ar.groupby(['order'])['year reported'].count().sort_values(ascending=False).plot(kind='barh',figsize=(12,10),xlabel='order of invasive arthropods',
                                                            title='Invasive arthropods in the US, by phylum',color="lightcoral",
                                                            ylabel="number invasive arthropods in order",width=1)
plt.savefig('Invasive arthropods in the US, summed by order.png')



In [ ]:
#the invasives data spans a long long time. how does only recent data look?
recent=invasives[invasives["year reported"]>=2000]
recar=recent[recent["phylum"]=="Arthropoda"]
recar.groupby(['order'])['year reported'].count().sort_values(ascending=False)
recar.head()


In [ ]:
#jointplot all invasives
sns.jointplot(x='year reported', y='class', data=invasives, kind='hist',
             hue="phylum",
              height=16)
plt.savefig('Invasive species in the US, 2000-2022.png')


In [ ]:
#jointplot recent invasives
plt.title("Recent invasions (200-2022)")

ax=sns.jointplot(x='year reported', y='class', data=recent, kind='hist',
             hue="phylum",
              height=16)
#sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.savefig('Invasive species in the US, 2000-2022 2.png')


In [ ]:
#jointplot recent arthropod invasives
plt.title("Recent arthropod invasions (200-2022)")
sns.jointplot(x='year reported', y='order', data=recar, kind='hist',
             hue="class",
              height=16)

plt.savefig('Invasive arthropods in the US, 2000-2022.png')


In [ ]:
#which species are the most frequent recent invasives?
recent.groupby(['phylum'])['year reported'].count().sort_values(ascending=False).plot(kind='barh',figsize=(12,10),xlabel='phylum of invasive species',
                                                            title='Invasive species in the US, by phylum',color="cadetblue",
                                                            ylabel="number invasive species in phylum",width=1)
plt.savefig('Invasive species in the US 2000-2022, summed by phylum.png')



In [ ]:
#which arthropods are the most frequent recent invasives?
ar.groupby(['order'])['year reported'].count().sort_values(ascending=False).plot(kind='barh',figsize=(12,10),xlabel='order of invasive arthropods',
                                                            title='Invasive arthropods in the US, by phylum',color="lightcoral",
                                                            ylabel="number invasive arthropods in order",width=1)
plt.savefig('Invasive arthropods in the US, summed by order.png')



<h3>An invasive pest: the soybean aphid, where did it come from, and when?</h3>  

These data describe the current distribution of the soybean aphid in the US
Source CABI Plant Pest Distribution database: https://www.cabi.org/isc/datasheet/6203#todistributionDatabaseTable

In [ ]:
#clean up soyaphiddist
#change data types as needed
soyaphiddist["Status"] = soyaphiddist['Status'].astype('bool')
soyaphiddist.dtypes
soyaphiddist.head()
soyaphiddist.columns


In [ ]:
#generate missing counts and heat maps to assess missing data
print("soyaphiddist",sns.heatmap(soyaphiddist.isnull(),cbar=True,vmin=0,vmax=.3))
print(soyaphiddist.isnull().sum())
print(soyaphiddist.dtypes)
print(soyaphiddist.head(20))
soyaphiddist.groupby(["country"])["state"].value_counts()



In [ ]:
#create separate dfs for mapping aphid distribution by country v by US state
#filter out rows where the relevant entity (state or country) is NaN
#Country
soyaphiddistcountry = soyaphiddist[soyaphiddist["country"].notna()]
soyaphiddistcountry = soyaphiddistcountry[['country', 'Status', 'year reported', 'country code']]
soyaphiddistcountry["year reported"]=soyaphiddistcountry["year reported"].astype('string')
#State
soyaphiddiststate=soyaphiddist[soyaphiddist["state"].notna()]
soyaphiddiststate = soyaphiddiststate[['state', 'Status', 'year reported', 'state code']]
soyaphiddiststate.rename(columns={"year reported":"Year soybean aphid first reported"},inplace=True)
soyaphiddiststate["Year soybean aphid first reported"].value_counts()


In [ ]:
#plot the soybean aphid's distribution across countries
df=pd.DataFrame({"Country":{0:"Cambodia",1:"China",2:"Indonesia",3:"Japan",
                           4:"South Korea",5:"Philippines",6:"Russia",7:"Thailand",
                           8:"Vietnam",9:"Australia",10:"United States",11:"Papua New Guinea"},
                 "Status of soybean aphid":{0:"Native to region",1:"Native to region",
                                            7:"Native to region",8:"Native to region",
                          3:"Native to region",4:"Native to region",5:"Established in region",
                           6:"Established in region",9:"Invasives detected in 1999",
                          10:"Invasives detected in 2001",2:"Established in region",
                                            11:"Established in region"},
                 "Country code":{0:"KHM",1:"CHN",2:"IDN",3:"JPN",4:"KOR",5:"PHL",
                                 6:"RUS",7:"THA",8:"VNM",9:"AUS",10:"USA",11:"PNG"}})

fig = px.choropleth(df, locations="Country",
                    locationmode="country names",color="Status of soybean aphid",
                    scope="world",width=700, height=500,
                    color_discrete_sequence= px.colors.sequential.Pinkyl_r,
                    title="The soybean aphid is native to China and is firmly established in many <br>nearby countries. Starting in 1999 it became an invasive pest on <br>new continents, <b>including North America</b>",
                    #labels=dict(native="Native region",established="Established in this region", 1999="Invasives detected in 1999", 2001="Invasives detected in 2001"),
                    #category_orders={"year reported": ["Invasives detected in 2001","Native region","Established in this region", "Invasives detected in 1999", ]},
                    template="plotly")
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.2,
    xanchor="right",
    x=1
))
fig.update_layout(font_family="Georgia",font_color="darkslategrey",font_size=14,)
fig.update_layout(showlegend=True)
fig.show()
#export image as HTML file and as PNG image
fig.write_html("countriesDistFig.html")
fig.write_image("countriesDistFig.png")

In [ ]:
#plot the soybean aphid's distribution across US states
#here using sequential scales as discrete sequences

fig = px.choropleth(soyaphiddiststate, locations="state code",
                    color_discrete_sequence= px.colors.sequential.OrRd_r,
                    locationmode="USA-states", scope="usa", color="Year soybean aphid first reported",
                    title="First detected in Minnesota in 1999, by 2004 the soybean aphid <br>was detected in <b>every major soybean-producing state</b>",
                    #legend="Year first reported <br>by soy growers", 
                    width=800, height=500,
                    category_orders={"Year soybean aphid first reported": ["1999", "2000", "2001", "2002","2004", "2005", "2008",]},
                    template="plotly_dark",
                   )
fig.update_layout(font_family="Courier",font_color="lightgrey",font_size=14,)
fig.update_layout(showlegend=True)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=-0.2,
    xanchor="right",
    x=1.1, 
    font=dict(
            family="Courier",
            size=11,
            color="white"
        ),
))
fig.show()
#export image as HTML file and as PNG image
fig.write_html("statesDistFig.html")
fig.write_image("statesDistFig.png")


<h3>One hundred years of soybean production in the United States</h3>

This is annual production data (1993-2022) for field crops in the US, which includes corn, soybean, grains, etc.
Source https://quickstats.nass.usda.gov/results/47D5D9C1-AD06-3A62-8B45-B09DEC5F7D8F

In [ ]:
#generate missing counts and heat maps to assess missing data
print("fieldcrops",sns.heatmap(fieldcrops.isnull(),cbar=True,vmin=0,vmax=.3))
print(fieldcrops.isnull().sum())
fieldcrops.head()

In [ ]:
#I'm only interested in soybean, this data set is too broad
fieldcrops.value_counts()

<h5>Soybean yield per acre and price per bushel</h5>   
This data set describes acres of soybean planted, yield per acre, and sale price from 1924-2022   

Source https://quickstats.nass.usda.gov/results/AA292938-D6B5-3B0D-B779-64A46043B700

In [ ]:
#generate missing counts and heat maps to assess missing data
print("nassyield",sns.heatmap(nassyield.isnull(),cbar=True,vmin=0,vmax=.3))
print(nassyield.isnull().sum())


In [ ]:
#these data are 100% soybean! 
nassyield.columns
nassyield["Measurement"].value_counts()


In [ ]:
#clean up nassyield
#rename columns to be more informative
nassyield=nassyield.rename(columns={"State":"Metric","Data Item":"Measurement",
                                       "Value": "Total"})
#retain only the relevant columns
nassyield=nassyield[["Year","Metric","Commodity","Measurement","Total"]]


In [ ]:
#clean up nassyield
#Rename the measurements
nassyield['Measurement'].replace(['SOYBEANS - YIELD, MEASURED IN BU / ACRE','SOYBEANS - PRICE RECEIVED, MEASURED IN $ / BU',
                                 "SOYBEANS - ACRES HARVESTED","SOYBEANS - ACRES PLANTED"],
                                 ['Yield in BU per acre','Price in $ per BU',"Harvested acres","Planted acres"]
                                 ,inplace=True)



In [ ]:
#clean up nassyield
#Total should be numeric
nassyield['Total']=pd.to_numeric(nassyield['Total'],errors='coerce')

#drop any rows where relevant sata are missing
nassyield=nassyield[nassyield["Total"].notna()] 
nassyield.dtypes
nassyield.head()

In [ ]:

nassyield.head()

In [ ]:
#clean up nassyield
#It would be easier to deal with the yield and price data separately since they are in different units
#I will create separate columns
#create a new df for each set of values, rename column
nassyieldbu=nassyield[(nassyield["Measurement"]=="Yield in BU per acre") & (nassyield["Total"].notna())]
nassyieldbu["Measurement"].value_counts()
nassyieldbu=nassyieldbu.rename(columns={"Total":"Bushels per acre"})
nassyieldprice=nassyield[(nassyield["Measurement"]=="Price in $ per BU") & (nassyield["Total"].notna())]
nassyieldprice=nassyieldprice.rename(columns={"Total":"Price per bushel"})

#add the new columns to the original df. can i merge?
#did sequential merges, inelegant but it worked
#But this solution left messy excess rows, 
#nassyield2=nassyield.merge(nassyieldprice,how="left",left_index=True,right_index=True)
#nassyield3=nassyield2.merge(nassyieldbu,how="left",left_index=True,right_index=True)

#So I'm trying another way
#there are 98 years with yield data, but only 26 years with price data. so i will retain na for price 
#so i can keep all the yield data. i should get at least 98 rows. YES!
nassyieldtmp=nassyieldbu.merge(nassyieldprice,how="left",on="Year")

#clean up the merged set
nassyield2=nassyieldtmp[["Year","Metric_x","Commodity_x","Bushels per acre","Price per bushel"]]
nassyield2=nassyield2.rename(columns={"Metric_x":"Metric","Commodity_x":"Commodity"})
print(nassyield2.head())
print(nassyield2.tail())



In [ ]:
#yield v price across years might be interesting
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Change in yield of soybeans over 100 years")
sns.scatterplot(x="Year", y="Bushels per acre", data=nassyield2)
#remove legend from sns plot
#plt.legend([],[], frameon=False)
plt.show()


In [ ]:
#yield around soy aphid time
recyield=nassyield2[(nassyield2["Year"]>1995) & (nassyield2["Year"]<2012)]



In [ ]:
#yield around soy aphid time
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Change in yield of soybean from 1995 to 2012")
fig=sns.lineplot(x="Year", y="Bushels per acre",
                 data=recyield, 
                 color="orange"
                )
plt.show()
fig.savefig('Change in yield of soybean from 1995 to 2012.png')


In [ ]:
#yield over time looks very linear. look at regression
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Changes in soybean yield in the last 100 years")
fig=sns.lmplot(x="Year", y="Bushels per acre",data=nassyield2,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           line_kws={'color': 'red'},
             )
plt.show()
fig.savefig('Changes in soybean yield in the last 100 years.png')


In [ ]:
#yield v price 
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Change in price of soybeans over 26 years")
sns.scatterplot(x="Year", y="Price per bushel", data=nassyield2)
#remove legend from sns plot
#plt.legend([],[], frameon=False)
plt.show()


In [ ]:
#price over time fluctuates a lot. look at regression
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Changes in soybean price since 1995")
fig=sns.lmplot(x="Year", y="Price per bushel",data=nassyield2,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           line_kws={'color': 'red'},order=3,
             )
plt.show()
fig.savefig('Changes in soybean price since 1995.png')


In [ ]:
#regression of price and yield
fig, ax = plt.subplots(figsize=(10,8))
plt.title("The relationship between soybean price and yield")
fig=sns.lmplot(x="Bushels per acre", y="Price per bushel",data=nassyield2,
           fit_reg=True, n_boot=10000, x_ci="sd",ci=20,scatter_kws={"s": 80},
           line_kws={'color': 'red'},order=2,
             )
plt.show()
fig.savefig('The relationship between soybean price and yield.png')


<h5>These are the annual costs and returns data for soybean grown in the US</h5>  

The data were split into "historical" vs "recent." I merged the two data sets before importing.  

Source historical: https://www.ers.usda.gov/data-products/commodity-costs-and-returns/commodity-costs-and-returns/#Historical%20Costs%20and%20Returns:%20Soybeans  

Source recent: https://www.ers.usda.gov/data-products/commodity-costs-and-returns/commodity-costs-and-returns/#Recent%20Cost%20and%20Returns


In [ ]:
#generate missing counts and heat maps to assess missing data
print("soycostreturn",sns.heatmap(soycostreturn.isnull(),cbar=True,vmin=0,vmax=.3))
print(soycostreturn.isnull().sum())
soycostreturn.head()

In [ ]:
soycostreturn.head()

In [ ]:
#clean up soycostreturn
#rename columns to be more informative
soycostreturn=soycostreturn.rename(columns={"Gross value of production":"gross value","Net value of production":"net value",
                                       "Chemicals": "chemical costs", "Hired labor":"labor costs",
                                            "Price (dollars per bushel at harvest)":"price per bushel",
                                            "Yield (bushels per planted acre)":"yield per acre"})
#retain only the relevant columns
soycostreturn=soycostreturn[["Year","gross value","net value","chemical costs","labor costs","price per bushel","yield per acre"]]
soycostreturn.head()
soycostreturn.dtypes



In [ ]:
#change in costs and value
#fig, ax = plt.subplots(figsize=(10,8))
plt.title("The price of labor over time")
fig=sns.jointplot(x="Year", y="labor costs", data=soycostreturn, kind='reg',
             #hue="Year",
              color="green",
              height=8)

sns.jointplot(x="labor costs", y="yield per acre", data=soycostreturn, kind='reg',
             #hue="Year",
              color="pink",

              height=8)

fig.savefig('Change in price of labor.png')


In [ ]:
#change in costs and value
#fig, ax = plt.subplots(figsize=(10,8))
#plt.title("The price of labor over time")
sns.jointplot(x="Year", y="labor costs", data=soycostreturn, kind='reg',
             #hue="Year",
              color="goldenrod",
              height=8)

sns.jointplot(x="labor costs", y="yield per acre", data=soycostreturn, kind='reg',
             #hue="Year",
              color="orange",
              height=8)

#plt.savefig('Change in price of labor.png')


In [ ]:
#change in costs and value
#fig, ax = plt.subplots(figsize=(10,8))
plt.title("The price of agricultural chemical")
sns.jointplot(x="Year", y="chemical costs", data=soycostreturn, kind='reg',
             #hue="Year",
              color="salmon",
              height=8)
plt.savefig('price of ag chemicals.png')


In [ ]:
#change in costs and value
#fig, ax = plt.subplots(figsize=(10,8))
plt.title("The price of chemicals relative to the net value of soybean")
sns.jointplot(x="net value", y="chemical costs", data=soycostreturn, kind='reg',
             #hue="Year",
              color="goldenrod",
              height=8)

plt.savefig('price of chemicals relative to the net value of soybean.png')


This data set shows the percent of US soybean treated with insecticides or herbicides   

source https://www.ers.usda.gov/publications/pub-details/?pubid=43855

In [ ]:
#generate missing counts and heat maps to assess missing data
print("pctreated",sns.heatmap(pesticides21crops.isnull(),cbar=True,vmin=0,vmax=.3))
print(pctreated.isnull().sum())
pctreated.columns
pctreated["Year"].value_counts()

In [ ]:
#set year as index to make lineplots easier
pctreated["Year"].is_unique
pctreated=pctreated.set_index("Year")


In [ ]:
#change in treated area
sns.set_style("darkgrid")
fig, ax = plt.subplots(figsize=(10,6))
plt.title("The percent of US soybean treated with insecticides")
#fig=sns.lineplot(data=pctreated,color="pink",markers=True,)
fig=sns.lineplot(data=pctreated,x="Year",y="Insecticides",color="pink")


plt.savefig('percent of US soybean treated with insecticides.png')


<h4>Pesticide use in 21 selected crops, 1960-2008</h4>   
This is data on acres planted, pesticide use, costs, and returns summed across 21 Selected Crops from 1960-2008  

The crops included are Apples, Barley, Corn, Cotton, Grapefruit, Grapes, Lemons, Lettuce, Oranges, Peaches, Peanuts, Pears, Pecans, Potatoes, Rice, Sorghum, Soybeans, Sugarcane, Sweetcorn, Tomatoes, and Wheat  

Source https://www.ers.usda.gov/publications/pub-details/?pubid=43855

In [ ]:
#generate missing counts and heat maps to assess missing data
print("pesticides21crops",sns.heatmap(pesticides21crops.isnull(),cbar=True,vmin=0,vmax=.3))
print(pesticides21crops.isnull().sum())


In [ ]:
#change in costs and value
fig, ax = plt.subplots(figsize=(10,8))
plt.title("Changes in the amount of pesticides used, 1940-2020")
g=sns.jointplot(x="Year", y="Total pounds pesticide use (million)", data=pesticides21crops, kind='kde',
             #hue="Year",
              color="red",
              height=10)
g.set_axis_labels('Year', 'Pounds of pesticides used (in millions)', fontsize=20)
plt.savefig('Changes in the amount of pesticides used, 1940-2020.png')


In [ ]:
#change in pesticide use
fig, ax = plt.subplots(figsize=(15,15))
plt.title("Changes in the amount of pesticides used")

sns.barplot(y="Year", x="Pesticides per acre (average)", data=pesticides21crops,
            #hue="Year",
            orient = 'h',
           )

#fig.savefig('2Changes in the amount of pesticides used.png')


<h2>Some observations</h2>
**chemical insecticide use**  
Over the past 40+ years, chemical pesticide use has increased but there are many dips and spikes.
The use of one neonicitinoid is increasing, another peaked ~2012 but use has remained steady
All ryanoids are increasing 
Two pyrethroids were increasing but in ~2015 the began to decline
One organophosphate was increasing until ~2012, now decreasing
All carbamates are decreasing
diacylhydrazine use jumps around, currently decreasing but year-to-year variation is high

**non-chemical pest control methods**   
Data from 2012 to the present show stable use of most methods. Only the "suppression" approach, which includes:  

pesticides with different mechanisms of action used to keep pest from becoming resistant to pesticides    
biological pesticides applied                                                                             
ground covers, mulches, or other physical barriers maintained                                             
buffer strips or border rows maintained to isolate organic from non organic crops                         
scouting data compared to published information to assist decisions                                       
beneficial organisms applied or released      
showed an increase, and this was mainly seen in "pesticides with different mechanisms..." which hardly counts as nonchemical in my opinion!   

**soybean price and yield**  
While yield has increased in a linear fashion for the last 100 years, price has not.  
From ~2000 to 2012 price increased steadily, since then it has been lower but steady, but with a jump in 2021.  
Price does not appear to reflect yield (eg, when productivity is lower, price does not increase). My price data are not corrected for current value of $1, but in terms of seeing year-to-year patterns I don't think that matters.  
Labor costs have not increased at the same rate as yield and net profits.  
The cost of chemicals *has* increased at about the same rate as yield and net profit.  

**The role of invasive pests**  
Although invasive pests are thought to be a major reason for growers to increase their use of pesticides, in fact I found no clear relationship between the detection of a new soy pest, the productivity of US soy, and the use of chemical pesticides.

**All conclusions are preliminary!**




<h4>Notes to self as I worked through the project</h4>
Document every data set as you obtain it, it's dumb to have to figure out where a data set came from when you are the one who collected it.

Some viz approaches  
#use seaborn to take a fast look at relationships between factors
#sns.pairplot(insecticidesbyyear,hue="Insecticide class")
#sns.displot(insecticidesbyyear["Fraction of acres treated with this insecticide"])
#insecticidesbyyear.groupby(["Year"])["Fraction of acres treated with this insecticide"].count().plot(kind='barh')
#insecticidesbyyear.groupby(["Year","Insecticide class"])["Fraction of acres treated with this insecticide"].mean().plot(kind='barh')
#sns.swarmplot(x="Year", y="Fraction of acres treated with this insecticide",data=insecticidesbyyear,hue="Insecticide cl



Other visualization approaches
#regplot
sns.regplot(x="Year", y="Fraction of acres treated with this insecticide",
               data=insecticidesbyyear,color="tab:red",x_estimator=np.mean);
               
#histogram
py.hist(column="Fraction of acres treated with this insecticide",
                        by="Year",
                        grid=False,figsize=(20, 10), legend=False,
                        color='tab:red')

**Plotly colors**   

These are named CSS colors, should work with Matplotlib, seaborn, etc as well as plotly  

![Image of plotly named colors](https://i.stack.imgur.com/xRwWi.png)

My notes on trying to get separate columns for yield and price    

#creates new col
df.loc[:,'D'] = df['B']

#creates new df based on conditional filter
df3 = df[df[‘Borough’] == Queens) & (df[Year] == 2019)]

nassyield.loc[:,"Yield in BU per acre"]=nassyield[nassyield["Measurement"]
nassyield.loc[:,"Yield in BU per acre"]=nassyield["Measurement"]

dftest=nassyield[nassyield.loc["Yield in BU per acre"]
                 
nassyieldbu=nassyield[(nassyield["Measurement"]=="Yield in BU per acre") & (nassyield["Total"].notna())]

#make a new col in original df based on conditional
nassyield["Yield"]=nassyield["Total"][(nassyield["Measurement"]=="Yield in BU per acre") & (nassyield["Total"].notna())]

#create a new df for each set of values based on conditional
nassyieldbu=nassyield[(nassyield["Measurement"]=="Yield in BU per acre") & (nassyield["Total"].notna())]               
 

In [ ]:
#To see plotly colors
fig1 = px.colors.qualitative.swatches()
fig1.show()
#To see a list of named built-in continuous color scales
from textwrap import wrap
named_colorscales = px.colors.named_colorscales()
print("##Plotly's named built-in continuous color scales","\n","\n".join(wrap("".join('{:<12}'.format(c) for c in named_colorscales), 96)))
##To see named built-in sequential color scales
fig2 = px.colors.sequential.swatches_continuous()
fig2.show()
##To see named built-in diverging color scales
fig3 = px.colors.diverging.swatches_continuous()
fig3.show()
##To see named built-in cyclical color scales
fig4 = px.colors.cyclical.swatches_cyclical()
fig4.show()
fig5 = px.colors.cyclical.swatches_continuous()
fig5.show()
#To see plotly themes
print("\n","\n""##Plotly's built-in templates")
ply.templates
